# TECHIN 509: Bigram Melody Generator

This project will use Python and a Bigram model to generate musical melodies, as specified in the `README.md` file.

We will follow modular programming principles, splitting the program into several clear functions.

In [ ]:
import random

## Part 1 & 2: Data Preparation

First, we define the sample dataset mentioned in the `README.md`.

This dataset is a `list of lists`, where each sub-list represents a single melody.

In [ ]:
# Our sample dataset (based on the README examples)
melody_dataset = [
    ['E4', 'F#4', 'G4', 'A4', 'B4', 'A4', 'G4', 'F#4', 'E4'],
    ['C4', 'D4', 'E4', 'C4'],
    ['C4', 'E4', 'G4', 'E4', 'C4'],
    ['G4', 'A4', 'G4', 'F#4', 'E4', 'D4', 'E4'],
    ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5']
]

## Part 5 (Clean Up): Melody Preprocessing

As required by Part 5 of the `README.md`, we need to add **start (`^`)** and **end (`$`)** tokens. This is excellent practice, as it allows the model to "learn" how melodies begin and end.

We'll define the `preprocess_melodies` function to handle this.

In [ ]:
def preprocess_melodies(melodies):
    """
    Adds start ('^') and end ('$') tokens to each melody in the dataset.

    Args:
        melodies (list): The list of melodies (list of lists).

    Returns:
        list: The list of melodies with tokens added.
    """
    processed = []
    for melody in melodies:
        # Add '^' to the beginning and '$' to the end of the list
        processed.append(['^'] + melody + ['$'])
    return processed

## Part 2: Build the Bigram Model

Now we'll write the `build_bigram_model` function. This function will iterate through all the processed melodies and count the occurrences of each "current note" to "next note" transition.

The model will use a nested dictionary structure:
`{ 'current_note': { 'next_note_1': count, 'next_note_2': count } }`

In [ ]:
def build_bigram_model(processed_melodies):
    """
    Builds a bigram model from the (processed) melody data.

    Args:
        processed_melodies (list): The list of melodies from preprocess_melodies.

    Returns:
        dict: The bigram model.
    """
    model = {}
    
    for melody in processed_melodies:
        # Iterate through each (current, next) pair in the melody
        # We stop at len(melody) - 1 to avoid an index out of bounds error
        for i in range(len(melody) - 1):
            current_note = melody[i]
            next_note = melody[i+1]
            
            # 1. Check if 'current_note' is already a key in the model
            if current_note not in model:
                model[current_note] = {}
            
            # 2. Check if 'next_note' is already in the entry for 'current_note'
            if next_note not in model[current_note]:
                model[current_note][next_note] = 0
            
            # 3. Increment the count
            model[current_note][next_note] += 1
            
    return model

## Part 3: Generate New Melodies

This is the most exciting part! We'll write the `generate_melody` function.

1.  Start with the `^` (start) token.
2.  Look up all possible next notes from `^` and their weights (counts) in the model.
3.  Use `random.choices()` to make a **weighted random selection**.
4.  Set this new note as the "current note" and repeat the process.
5.  If the `$` (end) token is chosen, or we hit the max length, the melody is complete.

In [ ]:
def generate_melody(model, max_length=20, forbid_three_repeats=True):
    """
    Generates a new melody using the bigram model.

    Args:
        model (dict): The model generated by build_bigram_model.
        max_length (int): Max length to prevent infinite loops.
        forbid_three_repeats (bool): If True, avoid repeating the same note
            three times in a row (optional Part 5 constraint).

    Returns:
        str: A space-separated string of notes.
    """
    melody = []
    current_note = '^'
    
    for _ in range(max_length):
        if current_note not in model:
            break
        
        transitions = model[current_note]
        next_notes = list(transitions.keys())
        weights = list(transitions.values())
        
        # Optional constraint: filter out a third repeated note when possible
        if forbid_three_repeats and len(melody) >= 2:
            filtered = [
                (note, weight)
                for note, weight in transitions.items()
                if not (melody[-1] == melody[-2] == note)
            ]
            if filtered:
                next_notes, weights = zip(*filtered)
                next_notes, weights = list(next_notes), list(weights)
        
        next_note = random.choices(next_notes, weights, k=1)[0]
        
        if next_note == '$':
            break
        
        melody.append(next_note)
        current_note = next_note
        
    return " ".join(melody)

## Part 4 & 5: Show Results and Analysis

Finally, we'll write two helper functions:

1.  `print_model_readable`: To print the model clearly, as requested in the `README.md`.
2.  `find_most_common_transition`: To iterate through the model and find the most-learned note transition.

In [ ]:
def print_model_readable(model):
    """
    Prints the Bigram model in a human-readable format.
    """
    print("--- Bigram Model ---")
    for current_note, transitions in model.items():
        # Format the dictionary for easier reading
        transition_str = ", ".join(f"'{note}': {count}" for note, count in transitions.items())
        print(f"{current_note} → {{{transition_str}}}")
    print("-" * 20)

def find_most_common_transition(model):
    """
    Finds the most common note transition (pair) in the dataset.
    """
    most_common_pair = (None, None)
    max_count = 0
    
    # Iterate through every entry in the model
    for current_note, transitions in model.items():
        for next_note, count in transitions.items():
            if count > max_count:
                max_count = count
                most_common_pair = (current_note, next_note)
                
    return most_common_pair, max_count

## Main Program: Executing All Steps

Now we'll chain all the functions together to run the full process and print the final results.

In [ ]:
def main(seed=42, samples=3, max_length=20):
    """Run the full melody generation pipeline with reproducible randomness."""
    random.seed(seed)

    print("Preprocessing data...")
    processed_data = preprocess_melodies(melody_dataset)

    print("Building bigram model...")
    bigram_model = build_bigram_model(processed_data)

    print_model_readable(bigram_model)

    print("--- Generated Melodies ---")
    for i in range(samples):
        new_song = generate_melody(
            bigram_model,
            max_length=max_length,
            forbid_three_repeats=True,
        )
        print(f"{i+1}. {new_song}")
    print("-" * 20)

    common_pair, count = find_most_common_transition(bigram_model)
    print("--- Analysis ---")
    print(f"Most common transition: {common_pair[0]} → {common_pair[1]} (Count: {count})")


if __name__ == "__main__":
    main()
